In [1]:
import keras

Using TensorFlow backend.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
keras.__version__

'2.3.1'

In [3]:
#Importing the dataset

from keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

In [4]:
len(train_data)

8982

In [5]:
len(test_data)

2246

In [6]:
train_data[10]

[1,
 245,
 273,
 207,
 156,
 53,
 74,
 160,
 26,
 14,
 46,
 296,
 26,
 39,
 74,
 2979,
 3554,
 14,
 46,
 4689,
 4329,
 86,
 61,
 3499,
 4795,
 14,
 61,
 451,
 4329,
 17,
 12]

In [7]:
#Data exploration
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
# Note that our indices were offset by 3
# because 0, 1 and 2 are reserved indices for "padding", "start of sequence", and "unknown".
decoded_newswire = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[20]])

In [8]:
decoded_newswire

"? leading u s farm state senators are seeking to ? into the ? ? trade bill a provision that would broaden eligibility requirements under the u s agriculture department's export enhancement program eep to include traditional buyers of u s farm products including the soviet union senate staff said under existing criteria usda can offer eep subsidies to ? export markets lost to competing nations' unfair trading practices senate agriculture committee chairman patrick leahy d vt is leading a group of farm state senators in an effort to broaden the criteria in such a way as to enable moscow to be eligible for the subsidies sources said the senators including senate finance committee chairman lloyd bentsen d tex max ? d ? david ? d ? john ? d ? and ? ? r miss also may ? into the trade bill a measure to shield pork producers and processors from canadian imports the measure sponsored by sen charles ? r iowa would clarify the definition of industry in determining whether or not imports were cau

In [9]:
train_labels[20]

11

In [10]:
# Data pre processing

import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

# Our vectorized training data
x_train = vectorize_sequences(train_data)
# Our vectorized test data
x_test = vectorize_sequences(test_data)

In [11]:
#One-Hot Encoding

def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results

# Our vectorized training labels
one_hot_train_labels = to_one_hot(train_labels)
# Our vectorized test labels
one_hot_test_labels = to_one_hot(test_labels)

In [12]:
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

In [13]:
# Building our network

from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

In [14]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
#Validation split

x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

In [31]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(4, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(partial_x_train,
          partial_y_train,
          epochs=20,
          batch_size=128,
          validation_data=(x_val, y_val))

Train on 7982 samples, validate on 1000 samples
Epoch 1/20
7982/7982 [==============================] - 0s 52us/step - loss: 3.3201 - accuracy: 0.1071 - val_loss: 2.7546 - val_accuracy: 0.4030
Epoch 2/20
7982/7982 [==============================] - 0s 45us/step - loss: 2.3162 - accuracy: 0.4331 - val_loss: 2.0916 - val_accuracy: 0.4440
Epoch 3/20
7982/7982 [==============================] - 0s 45us/step - loss: 1.7868 - accuracy: 0.4826 - val_loss: 1.7141 - val_accuracy: 0.4780
Epoch 4/20
7982/7982 [==============================] - 0s 46us/step - loss: 1.3942 - accuracy: 0.6113 - val_loss: 1.4491 - val_accuracy: 0.6580
Epoch 5/20
7982/7982 [==============================] - 0s 46us/step - loss: 1.1492 - accuracy: 0.7256 - val_loss: 1.3491 - val_accuracy: 0.6980
Epoch 6/20
7982/7982 [==============================] - 0s 46us/step - loss: 1.0069 - accuracy: 0.7611 - val_loss: 1.3200 - val_accuracy: 0.7030
Epoch 7/20
7982/7982 [==============================] - 0s 45us/step - loss: 0.910

In [35]:
#Generating predictions 

predictions = model.predict(x_test)

In [36]:
#Shape of the output

predictions[0].shape

(46,)

In [37]:
#SUm of all 46 probabilities for one input value is approximately 1
np.sum(predictions[0])

1.0